<a href="https://colab.research.google.com/github/riorio214/SNS-random/blob/main/steam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("trolukovich/steam-games-complete-dataset")

print("Path to dataset files:", path)

100%|██████████| 24.6M/24.6M [00:02<00:00, 11.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/trolukovich/steam-games-complete-dataset/versions/1


In [3]:
import os

# 다운로드한 경로의 파일 목록 확인
downloaded_files = os.listdir('/root/.cache/kagglehub/datasets/trolukovich/steam-games-complete-dataset/versions/1')
print(downloaded_files)

['steam_games.csv']


In [12]:
import kagglehub
import os
import pandas as pd

# CSV 파일 경로 설정
csv_file_path = '/root/.cache/kagglehub/datasets/trolukovich/steam-games-complete-dataset/versions/1/steam_games.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# 1. 'types' 열에서 'bundle' 값이 있는 행 삭제
df = df[df['types'] != 'bundle']

# 2. 'languages' 열에서 'Korean'이 포함되지 않은 행 삭제
df = df[df['languages'].str.contains('Korean', na=False)]

# 3. 'popular_tags' 열에서 'Multiplayer'가 포함되지 않은 행 삭제
df = df[df['popular_tags'].str.contains('Multiplayer', na=False)]

# /content 경로에 정제된 데이터 저장, UTF-8 인코딩
cleaned_file_path = '/content/steam_games_cleaned.csv'
df.to_csv(cleaned_file_path, index=False, encoding='utf-8')

print("전처리 완료:", cleaned_file_path)


전처리 완료: /content/steam_games_cleaned.csv


In [13]:
import kagglehub
import os
import pandas as pd

# CSV 파일 경로 설정
csv_file_path = '/content/steam_games_cleaned.csv'

# CSV 파일 UTF-8 인코딩으로 읽기
df = pd.read_csv(csv_file_path, encoding='utf-8')

print(df.head(5))


                                                 url types  \
0  https://store.steampowered.com/app/578080/PLAY...   app   
1  https://store.steampowered.com/app/601150/Devi...   app   
2  https://store.steampowered.com/app/477160/Huma...   app   
3  https://store.steampowered.com/app/527230/For_...   app   
4  https://store.steampowered.com/app/393080/Call...   app   

                                        name  \
0              PLAYERUNKNOWN'S BATTLEGROUNDS   
1                            Devil May Cry 5   
2                           Human: Fall Flat   
3                               For The King   
4  Call of Duty®: Modern Warfare® Remastered   

                                        desc_snippet  \
0  PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...   
1  The ultimate Devil Hunter is back in style, in...   
2  Human: Fall Flat is a quirky open-ended physic...   
3  For The King is a strategic RPG that blends ta...   
4  One of the most critically-acclaimed games in ...   

 

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 데이터 불러오기
df = pd.read_csv('/content/steam_games_cleaned.csv', encoding='utf-8')

# 숫자형 컬럼만 선택하여 NaN 값을 평균으로 대체
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# 특성과 레이블 설정
X = df.drop(columns=['genre', 'url'])  # 특성: 장르를 제외한 나머지 컬럼
y = df['genre']  # 레이블: 'genre' (장르)

# 범주형 변수를 더미 변수로 변환
X = pd.get_dummies(X)

# 학습용, 테스트용 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# SVM 모델 학습
svm_model = SVC(kernel='linear')  # 선형 커널을 사용하는 SVM
svm_model.fit(X_train, y_train)

# 예측
y_pred = svm_model.predict(X_test)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"SVM Accuracy: {accuracy * 100:.2f}%")


SVM Accuracy: 31.56%


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 데이터 불러오기
df = pd.read_csv('/content/steam_games_cleaned.csv', encoding='utf-8')

# 특성과 레이블 설정
X = df.drop(columns=['genre', 'url'])
y = df['genre']

# 텍스트 컬럼이 있다면 이를 숫자형으로 변환
X = pd.get_dummies(X)  # 범주형 변수를 더미 변수로 변환

# 학습용, 테스트용 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 로지스틱 회귀 모델 학습
logreg_model = LogisticRegression(max_iter=1000)  # 반복 횟수 늘려주기
logreg_model.fit(X_train, y_train)

# 예측
y_pred = logreg_model.predict(X_test)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Accuracy: {accuracy * 100:.2f}%")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score

# 데이터 불러오기
df = pd.read_csv('/content/steam_games_cleaned.csv', encoding='utf-8')

# 특성과 레이블 설정
X = df.drop(columns=['genre', 'url'])
y = df['genre']

# 텍스트 컬럼이 있다면 이를 숫자형으로 변환
X = pd.get_dummies(X)  # 범주형 변수를 더미 변수로 변환

# 학습용, 테스트용 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# XGBoost 모델 학습
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train, y_train)

# 예측
y_pred = xgb_model.predict(X_test)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost Accuracy: {accuracy * 100:.2f}%")
